In [23]:
import pandas as pd
from tabulate import tabulate
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [24]:
# from google.colab import drive
# drive.mount('/content/drive')

In [25]:
df = pd.read_csv('internet_service_churn.csv')

In [26]:
df.head()

,id,is_tv_subscriber,is_movie_package_subscriber,subscription_age,bill_avg,reamining_contract,service_failure_count,download_avg,upload_avg,download_over_limit,churn
0,15,1,0,11.95,25,0.14,0,8.4,2.3,0,0
1,18,0,0,8.22,0,NaN,0,0.0,0.0,0,1
2,23,1,0,8.91,16,0.00,0,13.7,0.9,0,1
3,27,0,0,6.87,21,NaN,1,0.0,0.0,0,1
4,34,0,0,6.39,0,NaN,0,0.0,0.0,0,1


In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72274 entries, 0 to 72273
Data columns (total 11 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   id                           72274 non-null  int64  
 1   is_tv_subscriber             72274 non-null  int64  
 2   is_movie_package_subscriber  72274 non-null  int64  
 3   subscription_age             72274 non-null  float64
 4   bill_avg                     72274 non-null  int64  
 5   reamining_contract           50702 non-null  float64
 6   service_failure_count        72274 non-null  int64  
 7   download_avg                 71893 non-null  float64
 8   upload_avg                   71893 non-null  float64
 9   download_over_limit          72274 non-null  int64  
 10  churn                        72274 non-null  int64  
dtypes: float64(4), int64(7)
memory usage: 6.1 MB


In [28]:
df.describe()

,id,is_tv_subscriber,is_movie_package_subscriber,subscription_age,bill_avg,reamining_contract,service_failure_count,download_avg,upload_avg,download_over_limit,churn
count,7.227400e+04,72274.000000,72274.000000,72274.000000,72274.000000,50702.000000,72274.000000,71893.000000,71893.000000,72274.000000,72274.000000
mean,8.463182e+05,0.815259,0.334629,2.450051,18.942483,0.716039,0.274234,43.689911,4.192076,0.207613,0.554141
std,4.891022e+05,0.388090,0.471864,2.034990,13.215386,0.697102,0.816621,63.405963,9.818896,0.997123,0.497064
min,1.500000e+01,0.000000,0.000000,-0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4.222165e+05,1.000000,0.000000,0.930000,13.000000,0.000000,0.000000,6.700000,0.500000,0.000000,0.000000
50%,8.477840e+05,1.000000,0.000000,1.980000,19.000000,0.570000,0.000000,27.800000,2.100000,0.000000,1.000000
75%,1.269562e+06,1.000000,1.000000,3.300000,22.000000,1.310000,0.000000,60.500000,4.800000,0.000000,1.000000
max,1.689744e+06,1.000000,1.000000,12.800000,406.000000,2.920000,19.000000,4415.200000,453.300000,7.000000,1.000000


In [29]:
df.duplicated().sum()

np.int64(0)

In [30]:
missing_values = df.isnull().sum()
missing_percentage = (missing_values / len(df)) * 100

table = zip(df.columns, missing_values, missing_percentage)
print(tabulate(table, headers=['Feature', 'Missing Values', 'Percentage']))

Feature                        Missing Values    Percentage
---------------------------  ----------------  ------------
id                                          0      0
is_tv_subscriber                            0      0
is_movie_package_subscriber                 0      0
subscription_age                            0      0
bill_avg                                    0      0
reamining_contract                      21572     29.8475
service_failure_count                       0      0
download_avg                              381      0.527161
upload_avg                                381      0.527161
download_over_limit                         0      0
churn                                       0      0


In [31]:
df['reamining_contract'].fillna(0, inplace=True)

In [32]:
df.dropna(subset=['download_avg'], inplace=True)
df.dropna(subset=['upload_avg'], inplace=True)

In [33]:
df['is_contract'] = df['reamining_contract'].apply(lambda x: 0 if x == 0 else 1)

In [34]:
df_with_id = df.copy()

df = df.drop('id', axis=1)

In [35]:
df_with_id.drop(columns=['service_failure_count'], inplace=True)

In [36]:
df_with_id['id'] = df_with_id['id'].apply(lambda x: f"C{x:07d}")

In [37]:
df_with_id.to_csv('cleaned_dataset_with_id.csv', index=False)

In [38]:
def detect_outliers_iqr(data):
    Q1 = data.quantile(0.25)
    Q3 = data.quantile(0.75)
    IQR = Q3 - Q1
    outliers = ((data < (Q1 - 1.5 * IQR)) | (data > (Q3 + 1.5 * IQR)))
    return outliers

# numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
# numeric_cols.remove("churn")

# numeric_cols = [
#     col for col in df.select_dtypes(include=[np.number]).columns
#     if col != "churn" and df[col].nunique() > 2
# ]

numeric_cols = [
    col for col in df.select_dtypes(include=[np.number]).columns
    if col != "churn"
    and df[col].nunique() > 2
    and col not in ['service_failure_count', 'download_over_limit']
]

outlier_flags = df[numeric_cols].apply(detect_outliers_iqr)

outlier_counts = outlier_flags.sum()
outlier_counts[outlier_counts > 0]

subscription_age      3203
bill_avg              3361
reamining_contract      15
download_avg          3473
upload_avg            4864
dtype: int64

In [39]:
def clip_outliers_iqr(column):
    Q1 = column.quantile(0.25)
    Q3 = column.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return column.clip(lower=lower_bound, upper=upper_bound)

df[numeric_cols] = df[numeric_cols].apply(clip_outliers_iqr)

post_clip_outlier_flags = df[numeric_cols].apply(detect_outliers_iqr)
post_clip_outlier_counts = post_clip_outlier_flags.sum()
post_clip_outlier_counts[post_clip_outlier_counts > 0]

Series([], dtype: int64)

In [40]:
df[numeric_cols].describe()

,subscription_age,bill_avg,reamining_contract,download_avg,upload_avg
count,71893.000000,71893.000000,71893.000000,71893.000000,71893.000000
mean,2.404109,18.093451,0.497574,39.959870,3.236150
std,1.879504,8.290427,0.665630,40.299303,3.375571
min,-0.020000,0.000000,0.000000,0.000000,0.000000
25%,0.940000,13.000000,0.000000,6.700000,0.500000
50%,1.980000,19.000000,0.000000,27.800000,2.100000
75%,3.300000,22.000000,1.030000,60.500000,4.800000
max,6.840000,35.500000,2.575000,141.200000,11.250000


In [41]:
df.head()

,is_tv_subscriber,is_movie_package_subscriber,subscription_age,bill_avg,reamining_contract,service_failure_count,download_avg,upload_avg,download_over_limit,churn,is_contract
0,1,0,6.84,25.0,0.14,0,8.4,2.3,0,0,1
1,0,0,6.84,0.0,0.00,0,0.0,0.0,0,1,0
2,1,0,6.84,16.0,0.00,0,13.7,0.9,0,1,0
3,0,0,6.84,21.0,0.00,1,0.0,0.0,0,1,0
4,0,0,6.39,0.0,0.00,0,0.0,0.0,0,1,0


In [42]:
df.rename(columns={'reamining_contract': 'remaining_contract'}, inplace=True)
df.head()

,is_tv_subscriber,is_movie_package_subscriber,subscription_age,bill_avg,remaining_contract,service_failure_count,download_avg,upload_avg,download_over_limit,churn,is_contract
0,1,0,6.84,25.0,0.14,0,8.4,2.3,0,0,1
1,0,0,6.84,0.0,0.00,0,0.0,0.0,0,1,0
2,1,0,6.84,16.0,0.00,0,13.7,0.9,0,1,0
3,0,0,6.84,21.0,0.00,1,0.0,0.0,0,1,0
4,0,0,6.39,0.0,0.00,0,0.0,0.0,0,1,0


In [43]:
df.to_csv('cleaned_dataset.csv', index=False)